# **LoopsResultsExploration**

## **1. Importing and First Proccesing**

In [10]:
import pandas as pd
from DataCleaning import *

pd.options.display.max_columns = 50

In [11]:
raw_data = pd.read_excel(cleaning_config['raw_data_path'])
raw_data = clean_data(raw_data)

In [12]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5071 entries, 7 to 10586
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   subject                 5071 non-null   object
 1   step_num                5071 non-null   int64 
 2   uid                     5071 non-null   int64 
 3   trial_start_time        5071 non-null   int64 
 4   rt                      5071 non-null   int32 
 5   response                5071 non-null   object
 6   loop_step               5071 non-null   int32 
 7   trial_order             5071 non-null   int64 
 8   var_mapping             5071 non-null   object
 9   trial_set               5071 non-null   int64 
 10  trial                   5071 non-null   int64 
 11  core_program            5071 non-null   int64 
 12  variant                 5071 non-null   object
 13  step_id                 5071 non-null   int64 
 14  text1                   5071 non-null   object
 15  res

In [13]:
raw_data.head()

,subject,step_num,uid,trial_start_time,rt,response,loop_step,trial_order,var_mapping,trial_set,trial,core_program,variant,step_id,text1,response_needed,expected_response,is_loop,loop_type,loop_type_switch,n_iterations,n_loop_lines,step_type,prev_loop_type,correct,same_as_prev_loop_type
7,107B,7,30,38002,818,2,1,1,1=a;2=i,2,4,2,FWWWWFFW,3,while a >= 2:\n a /= 2,True,"[4, 2, 1]",True,while,True,3,1,loop,0,False,False
8,107B,8,30,38811,302,1,2,1,1=a;2=i,2,4,2,FWWWWFFW,3,while a >= 2:\n a /= 2,True,"[4, 2, 1]",True,while,True,3,1,loop,0,False,False
11,107B,11,31,43376,69,5,1,1,1=a;2=i,2,4,2,FWWWWFFW,4,while a <= 7:\n a += 2,True,"[3, 5, 7, 9]",True,while,False,4,1,loop,0,False,False
12,107B,12,31,44123,237,7,2,1,1=a;2=i,2,4,2,FWWWWFFW,4,while a <= 7:\n a += 2,True,"[3, 5, 7, 9]",True,while,False,4,1,loop,0,False,False
13,107B,13,31,44963,327,9,3,1,1=a;2=i,2,4,2,FWWWWFFW,4,while a <= 7:\n a += 2,True,"[3, 5, 7, 9]",True,while,False,4,1,loop,0,False,False


## **2. Exploring Outliers**

### 2.1. Response-Time Between Subjects

In [60]:
response_times = raw_data[['subject', 'step_num', 'rt']].copy()
response_times.drop_duplicates(inplace=True)
response_time_agg = response_times[['rt', 'subject']].groupby('subject').agg(['std', 'mean'])
response_time_agg.columns = ['_'.join(col) for col in response_time_agg.columns.values]
response_time_agg.sort_values(by='rt_std')

,rt_std,rt_mean
subject,,
107B,1107.481888,1337.406818
104A,1154.958519,1452.411364
109B,1205.228505,1131.595455
108A,1257.528715,1740.395455
104B,1451.739484,1394.095455
105B,1544.058469,2513.627273
101B,1572.542826,2130.194805
106A,1602.449832,1917.463636
102B,1782.107346,2276.977273


In [66]:
mean_q3, mean_q1 = response_time_agg['rt_mean'].quantile([0.75, 0.25])
mean_iqr = mean_q3 - mean_q1

def is_outlier(mean):
    return (mean_q1 - mean) / mean_iqr >= 1.5 or (mean - mean_q3) / mean_iqr >= 1.5

mean_outlier_mask = response_time_agg['rt_mean'].apply(is_outlier)
mean_outliers = response_time_agg[mean_outlier_mask].index

if mean_outliers.size > 0:
    print(f'Seems that: {list(mean_outliers)} are outliers in terms of mean response time within subject.')
else:
    print("No Outliers detected! at least in terms of mean response time within subject.")

No Outliers detected.


In [67]:
std_q3, std_q1 = response_time_agg['rt_std'].quantile([0.75, 0.25])
std_iqr = std_q3 - std_q1

def is_outlier(std):
    return (std_q1 - std) / std_iqr >= 1.5 or (std - std_q3) / std_iqr >= 1.5

std_outlier_mask = response_time_agg['rt_std'].apply(is_outlier)
std_outliers = response_time_agg[std_oulier_mask].index
if std_outliers.size > 0:
    print(f'Seems that: {list(std_outliers)} are outliers in terms of response time variance within subject.')
else:
    print("No Outliers detected! at least in terms of response time variance within subject.")

Seems that: ['103B'] are outliers in terms of response time variance within subject.


#### *It seems that in terms of response time variance within subject, **103B** is an outlier, and **108B** is arguably also an outlier.*

In [71]:
print(f"103B's grade: {(response_time_agg.loc['103B', 'rt_std'] - std_q3) / std_iqr}")
print(f"108B's grade: {(response_time_agg.loc['108B', 'rt_std'] - std_q3) / std_iqr}")

103B's grade: 1.856285494638352
108B's grade: 1.470637240522526


#### *I would not filter 103B and 108B out, because they are not so extreme.*